# 🚀 FoadsIA - Backend Production (T4 GPU)

Este notebook ejecuta el ecosistema completo de **FoadsIA** en Google Colab con GPU T4.

## 📋 Instrucciones
1. Asegúrate de tener GPU habilitada: `Runtime > Change runtime type > T4 GPU`
2. Ejecuta las celdas en orden
3. Copia la URL pública generada para conectar tu frontend

## ⚡ Última Actualización
- ✅ Integración con **Master Loader Script**
- ✅ Descarga y Verificación Automática de Modelos
- ✅ Soporte SDXL Lightning (Hiperrealismo)
- ✅ Soporte Whisper (Subtítulos)

---

In [ ]:
# @title 🛠️ 1. Configuración del Entorno

import os
import sys
import subprocess
from pathlib import Path

# Configuración
REPO_URL = "https://github.com/Juanpalojime/FoadsIA.git"
BASE_DIR = Path("/content")
REPO_DIR = BASE_DIR / "FoadsIA"
BACKEND_DIR = REPO_DIR / "backend"

def run_command(cmd, description, silent=False):
    """Ejecuta un comando con manejo de errores."""
    print(f"⚙️  {description}...")
    try:
        if silent:
            result = subprocess.run(
                cmd, 
                shell=True, 
                check=True,
                stdout=subprocess.DEVNULL,
                stderr=subprocess.DEVNULL
            )
        else:
            result = subprocess.run(cmd, shell=True, check=True, capture_output=True, text=True)
        print(f"✅ {description} completado")
        return True
    except subprocess.CalledProcessError as e:
        print(f"❌ Error en {description}: {e}")
        if not silent and hasattr(e, 'stderr'):
            print(f"   Detalles: {e.stderr[:200]}")
        return False

# Resetear al directorio base
os.chdir(BASE_DIR)
print(f"📁 Directorio base: {os.getcwd()}\n")

# Clonar o actualizar repositorio
print("📡 Configurando repositorio...")
if not REPO_DIR.exists():
    if run_command(f"git clone {REPO_URL}", "Clonando repositorio"):
        print("✅ Repositorio clonado exitosamente\n")
    else:
        raise Exception("No se pudo clonar el repositorio")
else:
    os.chdir(REPO_DIR)
    if run_command("git pull origin master", "Actualizando repositorio"):
        print("✅ Repositorio actualizado\n")

# Verificar estructura del proyecto
if not BACKEND_DIR.exists():
    raise Exception(f"❌ No se encontró el directorio backend en {REPO_DIR}")

os.chdir(BACKEND_DIR)
print(f"✅ Directorio de trabajo: {os.getcwd()}\n")

# Instalar dependencias del sistema
print("📦 Instalando dependencias del sistema...")
system_packages = "ffmpeg libsm6 libxext6 libgl1"
run_command(
    f"apt-get update -qq && apt-get install -y -qq {system_packages}",
    "Dependencias del sistema",
    silent=True
)

# Instalar dependencias de Python
print("\n🐍 Instalando dependencias de Python...")
if (BACKEND_DIR / "requirements.txt").exists():
    run_command(
        "pip install -q -r requirements.txt",
        "Instalación de requirements.txt",
        silent=True
    )
else:
    print("⚠️  No se encontró requirements.txt")

# Asegurar dependencias críticas
print("\n🔧 Verificando dependencias críticas...")
critical_packages = "flask flask-socketio flask-cors eventlet pyngrok torch torchvision diffusers huggingface_hub openai-whisper edge-tts"
run_command(
    f"pip install -q {critical_packages}",
    "Dependencias críticas",
    silent=True
)

# Verificar GPU
print("\n🎮 Verificando disponibilidad de GPU...")
try:
    import torch
    if torch.cuda.is_available():
        gpu_name = torch.cuda.get_device_name(0)
        gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
        print(f"✅ GPU DETECTADA: {gpu_name}")
        print(f"   Memoria: {gpu_memory:.1f} GB")
    else:
        print("⚠️  GPU no disponible - el modelo correrá en CPU (muy lento)")
except Exception as e:
    print(f"❌ Error al verificar GPU: {e}")

print("\n" + "="*60)
print("🎉 CONFIGURACIÓN COMPLETADA - Listo para preparar modelos")
print("="*60 + "\n")

In [ ]:
# @title ⬇️ 1.5 Preparar Modelos (Master Loader)

import os
from pathlib import Path

# Validar directorio
BACKEND_DIR = Path("/content/FoadsIA/backend")
os.chdir(BACKEND_DIR)

print("🤖 EJECUTANDO SCRIPT MAESTRO DE MODELOS...\n")
if (BACKEND_DIR / "master_loader.py").exists():
    !python master_loader.py
else:
    print("❌ No se encontró master_loader.py. Por favor actualiza el repositorio (paso 1).")


In [ ]:
# @title 🚀 2. Ejecutar Servidor Backend

import os
import time
import threading
from pathlib import Path
from pyngrok import ngrok

# Token de ngrok (obtén el tuyo en https://dashboard.ngrok.com)
AUTH_TOKEN = "2yHQiBeYhFdbJSiK31054jtsKkw_54yvtD5Cs9mK2yhFgQ2j" #@param {type:"string"}
PORT = 5000 #@param {type:"integer"}

# Validar directorio
BACKEND_DIR = Path("/content/FoadsIA/backend")
os.chdir(BACKEND_DIR)

# Configurar ngrok
if AUTH_TOKEN:
    try:
        ngrok.set_auth_token(AUTH_TOKEN)
        print("✅ Token de Ngrok configurado")
    except Exception as e:
        print(f"⚠️  Error al configurar token: {e}\n")
else:
    print("⚠️  No se proporcionó token de ngrok\n")

# Limpiar túneles existentes
try:
    ngrok.kill()
    time.sleep(1)
except: pass

# Crear túnel
print(f"🌐 Iniciando túnel público en puerto {PORT}...")
try:
    tunnel = ngrok.connect(PORT, bind_tls=True)
    public_url = tunnel.public_url
    
    print("\n" + "="*70)
    print("🎉 SERVIDOR PÚBLICO ACTIVO")
    print("="*70)
    print(f"\n📡 URL del Backend: {public_url}")
    print(f"\n⚠️  IMPORTANTE: No cierres esta pestaña de navegador")
    print("="*70 + "\n")
    
except Exception as e:
    print(f"❌ Error al crear túnel: {e}")
    raise

# Iniciar servidor Flask
print("🚀 Iniciando servidor Flask...\n")
try:
    !python app.py
except KeyboardInterrupt:
    print("\n🛑 Servidor detenido.")
    ngrok.kill()
except Exception as e:
    print(f"\n❌ Error crítico: {e}")
    ngrok.kill()

---

## 🔧 Troubleshooting

- Si falla la descarga de modelos, simplemente **vuelve a ejecutar la celda 1.5**.
- Si el servidor se detiene inesperadamente, revisa si la GPU sigua asignada en `Runtime > View resources`.

**Desarrollado por FoadsIA Team** 🚀